In [1]:
import pymc as pm
import numpy as np
import arviz as az
from aesara.tensor import subtensor as st

%load_ext lab_black
%load_ext watermark

# Multinomial regression

Adapted from [Unit 7: NHANESmulti.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/NHANESmulti.odc)

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/paraguay.csv).

Associated lecture video: Unit 7 lesson 17

## Problem statement

The National Health and Nutrition Examination Survey (NHANES) is a program of studies designed to assess the health and nutritional status of adults and children in the United States. The survey is unique in that it combines interviews and physical examinations. 

Assume that N subjects select a choice form K categories. The i-th subject is characterized by 3 covariates x[i,1], x[i,2], and x[i,3]. Given the covariates, model the probability of a subject selecting the category k, k=1,...,K.


In [2]:
# data
# fmt: off
y = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1],
              [0, 0, 0, 1, 0]])

X =np.array([[2, 4, 9],
             [1, 5, 10],
             [1, 6, 14],
             [2, 4, 21],
             [2, 4, 22],
             [2, 6, 30],
             [3, 3, 33],
             [3, 2, 36],
             [3, 1, 40],
             [4, 1, 44]])
# fmt: on

In [3]:
y.shape

(10, 5)

In [4]:
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X_aug.shape

(10, 4)

In [5]:
with pm.Model() as m:
    y_data = pm.Data("y", y, mutable=False)
    X_data = pm.Data("X", X_aug, mutable=True)

    _beta = pm.Normal("_beta", 0, tau=0.1, shape=(4, 5))
    # first col in BUGS example is assigned 0
    # might be better to prepend a column of zeroes rather than overwrite
    beta = pm.Deterministic("beta", st.set_subtensor(_beta[:, 0], 0))
    eta = pm.math.exp(pm.math.dot(X_data, beta))
    p = eta / pm.math.sum(eta, axis=1)

    pm.Multinomial("likelihood", n=1, p=p, observed=y)

    trace = pm.sample(10000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [_beta]


Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 40 seconds.


In [11]:
X_new = np.array([1, 3, 3, 30]).reshape((1, 4))

with m:
    pm.set_data({"X": X_new})
    ppc = pm.sample_posterior_predictive(trace, predictions=True)

Sampling: [likelihood]


In [13]:
ppc

Inference data with groups:
	> predictions
	> predictions_constant_data

In [15]:
az.summary(ppc.predictions.mean(dim=("likelihood_dim_0")))

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
likelihood[0],0.008,0.050,0.0,0.0,0.000,0.000,36051.0,36068.0,1.0
likelihood[1],0.039,0.121,0.0,0.2,0.001,0.000,40391.0,39915.0,1.0
likelihood[2],0.120,0.210,0.0,0.6,0.001,0.001,41641.0,39797.0,1.0
likelihood[3],0.679,0.313,0.1,1.0,0.002,0.001,41428.0,37641.0,1.0
likelihood[4],0.154,0.229,0.0,0.7,0.001,0.001,42681.0,38502.0,1.0


In [16]:
az.summary(ppc.predictions.mean(dim=("likelihood_dim_0")))["mean"].sum()

1.0

In [8]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Thu Nov 03 2022

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

aesara: 2.8.7
aeppl : 0.0.38

numpy : 1.23.4
pymc  : 4.2.2
arviz : 0.13.0
aesara: 2.8.7

